## 사용 package 선언

In [1]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch import optim
from torch import nn

from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms

import random
from glob import glob
import pandas as pd
import numpy as np
from PIL import Image

In [166]:
################## YOLO 쓰기위한 저장
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [171]:
from utils.general import check_img_size, check_requirements, non_max_suppression, scale_coords

## 데이터관련 함수 정의 및 데이터셋 선언

In [2]:
from glob import glob

import pandas as pd
import numpy as np
import transformers
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms


def extract_day(file_name):
    day = int(file_name.split('.')[-2][-2:])
    return day


def make_day_array(image_pathes):
    day_array = np.array([extract_day(file_name) for file_name in image_pathes])
    return day_array


def make_image_path_array(root_path=None):
    if root_path is None:
        bc_directories = glob('./BC/*')
        lt_directories = glob('./LT/*')

    else:
        bc_directories = glob(root_path + 'BC/*')
        lt_directories = glob(root_path + 'LT/*')

    bc_image_path = []
    for bc_path in bc_directories:
        images = glob(bc_path + '/*.png')
        bc_image_path.extend(images)

    lt_image_path = []
    for lt_path in lt_directories:
        images = glob(lt_path + '/*.png')
        lt_image_path.extend(images)

    return bc_image_path, lt_image_path


def make_dataframe(root_path=None):
    bc_image_path, lt_image_path = make_image_path_array(root_path)
    bc_day_array = make_day_array(bc_image_path)
    lt_day_array = make_day_array(lt_image_path)

    bc_df = pd.DataFrame({'file_name': bc_image_path,
                          'day': bc_day_array})
    bc_df['species'] = 'bc'

    lt_df = pd.DataFrame({'file_name': lt_image_path,
                          'day': lt_day_array})
    lt_df['species'] = 'lt'

    total_data_frame = pd.concat([bc_df, lt_df]).reset_index(drop=True)

    return total_data_frame


def make_combination(length, species, data_frame):
    before_file_path = []
    after_file_path = []
    time_delta = []

    for i in range(length):
        sample = data_frame[data_frame['species'] == species].sample(2)
        after = sample[sample['day'] == max(sample['day'])].reset_index(drop=True)
        before = sample[sample['day'] == min(sample['day'])].reset_index(drop=True)

        before_file_path.append(before.iloc[0]['file_name'])
        after_file_path.append(after.iloc[0]['file_name'])
        delta = int(after.iloc[0]['day'] - before.iloc[0]['day'])
        time_delta.append(delta)

    combination_df = pd.DataFrame({
        'before_file_path': before_file_path,
        'after_file_path': after_file_path,
        'time_delta': time_delta,
    })

    combination_df['species'] = species

    return combination_df


class KistDataset(Dataset):
    def __init__(self, combination_df, is_test= None):
        self.combination_df = combination_df
        self.transform = transforms.Compose([
            transforms.Resize(640),
            transforms.ToTensor()
        ])
        self.is_test = is_test

    def __getitem__(self, idx):
        before_image = Image.open(self.combination_df.iloc[idx]['before_file_path'])
        after_image = Image.open(self.combination_df.iloc[idx]['after_file_path'])

        before_image = self.transform(before_image)
        after_image = self.transform(after_image)
        if self.is_test:
            return before_image, after_image
        time_delta = self.combination_df.iloc[idx]['time_delta']
        return before_image, after_image, time_delta

    def __len__(self):
        return len(self.combination_df)



## 모델선언

In [3]:
import torch
from torch import nn
from torchvision.models import mobilenet_v2

class CompareCNN(nn.Module):

    def __init__(self):
        super(CompareCNN, self).__init__()
        self.YOLO_net = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

    def forward(self, input):
        x = self.YOLO_net(input)
        return x


class CompareNet(nn.Module):

    def __init__(self):
        super(CompareNet, self).__init__()
        self.before_net = CompareCNN()
        self.after_net = CompareCNN()

    def forward(self, before_input, after_input):
        before = self.before_net(before_input)
        after = self.after_net(after_input)
        delta = before - after
        return delta


In [19]:
0.00001 == 1e-5

True

In [15]:
def seed_everything(seed): # seed 고정
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


seed_everything(2048)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
lr = 1e-5
epochs = 10
batch_size = 64
valid_batch_size = 64

model = CompareNet().to(device)
path = "/Users/hyunwoo/Desktop/open/train_dataset/"
total_dataframe = make_dataframe(root_path = path)
bt_combination = make_combination(10000, 'bc', total_dataframe)
lt_combination = make_combination(10000, 'lt', total_dataframe)

bt_train = bt_combination.iloc[:4500]
bt_valid = bt_combination.iloc[4500:]

lt_train = lt_combination.iloc[:4500]
lt_valid = lt_combination.iloc[4500:]

train_set = pd.concat([bt_train, lt_train])
valid_set = pd.concat([bt_valid, lt_valid])



train_dataset = KistDataset(train_set)
valid_dataset = KistDataset(valid_set)

optimizer = optim.Adam(model.parameters(), lr=lr)

train_data_loader = DataLoader(train_dataset,
                               batch_size=batch_size,
                               shuffle=True)

valid_data_loader = DataLoader(valid_dataset,
                               batch_size=valid_batch_size)

'''
for epoch in tqdm(range(epochs)):
    for step, (before_image, after_image, time_delta) in tqdm(enumerate(train_data_loader)):
        before_image = before_image.to(device)
        after_image = after_image.to(device)
        time_delta = time_delta.to(device)

        optimizer.zero_grad()
        logit = model(before_image, after_image)
        train_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - time_delta.float())) /
                      torch.LongTensor([batch_size]).squeeze(0).to(device))
        train_loss.backward()
        optimizer.step()

        if step % 15 == 0:
            print('\n=====================loss=======================')
            print(f'\n=====================EPOCH: {epoch}=======================')
            print(f'\n=====================step: {step}=======================')
            print('MAE_loss : ', train_loss.detach().cpu().numpy())
    valid_losses = []
    with torch.no_grad():
        for valid_before, valid_after, time_delta in tqdm(valid_data_loader):
            valid_before = valid_before.to(device)
            valid_after = valid_after.to(device)
            valid_time_delta = time_delta.to(device)


            logit = model(valid_before, valid_after)
            valid_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - valid_time_delta.float())) /
                          torch.LongTensor([valid_batch_size]).squeeze(0).to(device))
            valid_losses.append(valid_loss.detach().cpu())


    print(f'VALIDATION_LOSS MAE : {sum(valid_losses)/len(valid_losses)}')
    checkpoiont = {
        'model': model.state_dict(),

    }

    torch.save(checkpoiont, 'checkpoiont_128.pt')

'''

Using cache found in /Users/hyunwoo/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-12-15 torch 1.10.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /Users/hyunwoo/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-12-15 torch 1.10.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


"\nfor epoch in tqdm(range(epochs)):\n    for step, (before_image, after_image, time_delta) in tqdm(enumerate(train_data_loader)):\n        before_image = before_image.to(device)\n        after_image = after_image.to(device)\n        time_delta = time_delta.to(device)\n\n        optimizer.zero_grad()\n        logit = model(before_image, after_image)\n        train_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - time_delta.float())) /\n                      torch.LongTensor([batch_size]).squeeze(0).to(device))\n        train_loss.backward()\n        optimizer.step()\n\n        if step % 15 == 0:\n            print('\n=====================loss=======================')\n            print(f'\n=====================EPOCH: {epoch}=======================')\n            print(f'\n=====================step: {step}=======================')\n            print('MAE_loss : ', train_loss.detach().cpu().numpy())\n    valid_losses = []\n    with torch.no_grad():\n        for valid_before, valid_

In [5]:
total_dataframe

,file_name,day,species
0,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,20,bc
1,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,8,bc
2,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,9,bc
3,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,21,bc
4,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,23,bc
...,...,...,...
748,/Users/hyunwoo/Desktop/open/train_dataset/LT/L...,10,lt
749,/Users/hyunwoo/Desktop/open/train_dataset/LT/L...,4,lt
750,/Users/hyunwoo/Desktop/open/train_dataset/LT/L...,5,lt
751,/Users/hyunwoo/Desktop/open/train_dataset/LT/L...,11,lt


In [6]:
bt_combination = make_combination(5000, 'bc', total_dataframe)
lt_combination = make_combination(5000, 'lt', total_dataframe)



In [16]:
bt_combination

,before_file_path,after_file_path,time_delta,species
0,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,6,bc
1,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,19,bc
2,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,16,bc
3,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,5,bc
4,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,33,bc
...,...,...,...,...
9995,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,4,bc
9996,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,20,bc
9997,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,14,bc
9998,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,/Users/hyunwoo/Desktop/open/train_dataset/BC/B...,4,bc


In [185]:
check_img_size(before_image[0][0][0])

        0.66667, 0.65490, 0.65490, 0.64706, 0.63922, 0.62745, 0.62353, 0.59216, 0.45098, 0.41569, 0.47451, 0.51373, 0.54118, 0.56078, 0.56078, 0.57255, 0.57255, 0.56863, 0.57255, 0.56078, 0.54902, 0.54902, 0.54902, 0.54118, 0.53333, 0.53333, 0.52941, 0.52549, 0.50980, 0.50980, 0.51373, 0.50980, 0.50588, 0.50196,
        0.49804, 0.48235, 0.47843, 0.48235, 0.47843, 0.47059, 0.47843, 0.47451, 0.47059, 0.47843, 0.47059, 0.47451, 0.47451, 0.48235, 0.47843, 0.48627, 0.49020, 0.47843, 0.46667, 0.46667, 0.46667, 0.46275, 0.46275, 0.46275, 0.46667, 0.47843, 0.47059, 0.47059, 0.46667, 0.47059, 0.47843, 0.47843, 0.48235, 0.48235,
        0.47843, 0.47843, 0.47843, 0.47843, 0.47843, 0.48627, 0.49020, 0.49020, 0.48627, 0.49020, 0.49412, 0.49412, 0.49020, 0.49020, 0.49020, 0.49412, 0.50196, 0.49804, 0.49412, 0.49020, 0.49020, 0.49804, 0.49020, 0.49804, 0.49412, 0.49804, 0.49804, 0.50196, 0.50588, 0.49804, 0.49804, 0.50196, 0.50980, 0.51373,
        0.50980, 0.50196, 0.50196, 0.50588, 0.50980, 0.509

[32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,


In [146]:
test_set["r_path"][0]

'/Users/hyunwoo/Desktop/open/test_dataset/LT/1003/idx_LT_1003_00154.png'

In [ ]:
test_set = pd.read_csv('/Users/hyunwoo/Desktop/open/test_dataset/test_data.csv')
test_set['l_root'] = test_set['before_file_path'].map(lambda x: '/Users/hyunwoo/Desktop/open/test_dataset/' + x.split('_')[1] + '/' + x.split('_')[2])
test_set['r_root'] = test_set['after_file_path'].map(lambda x: '/Users/hyunwoo/Desktop/open/test_dataset/' + x.split('_')[1] + '/' + x.split('_')[2])
test_set['l_path'] = test_set['l_root'] + '/' + test_set['before_file_path'] + '.png' 
test_set['r_path'] = test_set['r_root'] + '/' + test_set['after_file_path'] + '.png'
test_dataset = KistDataset(test_set, is_test=True)
test_data_loader = DataLoader(test_dataset,
                               batch_size=64)


test_value = []
with torch.no_grad():
    for test_before, test_after in tqdm(test_data_loader):
        test_before = test_before.to(device)
        test_after = test_after.to(device)
        logit = model(test_before, test_after)
        value = logit.squeeze(1).detach().cpu().float()
        
        test_value.extend(value)

In [151]:
test_dataset.combination_df.iloc[0]['before_file_path']

'idx_LT_1003_00341'

In [154]:
test_set['l_path'][0]

'/Users/hyunwoo/Desktop/open/test_dataset/LT/1003/idx_LT_1003_00341.png'

In [160]:
test_dataset.combination_df.iloc[0]

idx                                                                 0
before_file_path                                    idx_LT_1003_00341
after_file_path                                     idx_LT_1003_00154
l_root               /Users/hyunwoo/Desktop/open/test_dataset/LT/1003
r_root               /Users/hyunwoo/Desktop/open/test_dataset/LT/1003
l_path              /Users/hyunwoo/Desktop/open/test_dataset/LT/10...
r_path              /Users/hyunwoo/Desktop/open/test_dataset/LT/10...
Name: 0, dtype: object